<a href="https://colab.research.google.com/github/G0nz4lo-4lvarez-H3rv4s/FakeNewsDetection/blob/main/Experimentaci%C3%B3n_detecci%C3%B3n_de_Fake_News.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip3 install huggingface_hub transformers datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 86 kB 3.7 MB/s 
     |████████████████████████████████| 4.4 MB 17.5 MB/s 
     |████████████████████████████████| 362 kB 69.7 MB/s 
     |████████████████████████████████| 596 kB 61.1 MB/s 
     |████████████████████████████████| 6.6 MB 52.7 MB/s 
     |████████████████████████████████| 1.1 MB 47.4 MB/s 
     |████████████████████████████████| 212 kB 67.7 MB/s 
     |████████████████████████████████| 140 kB 74.0 MB/s 
     |████████████████████████████████| 127 kB 77.9 MB/s 
     |████████████████████████████████| 144 kB 64.7 MB/s 
     |████████████████████████████████| 94 kB 3.1 MB/s 
     |████████████████████████████████| 271 kB 72.2 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: pyyaml
    Found existing 

In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer, TrainingArguments, Trainer, pipeline, DataCollatorWithPadding
from datasets import load_metric, load_dataset
import pandas as pd
import torch
import numpy as np
from huggingface_hub import notebook_login
notebook_login()

/usr/local/lib/python3.7/dist-packages/huggingface_hub/hf_api.py:587: FutureWarning: HfApi.login: This method is deprecated in favor of `set_access_token` and will be removed in v0.8.
  FutureWarning,


Login successful
Your token has been saved to /root/.huggingface/token
Authenticated through git-credential store but this isn't the helper defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub. Run the following command in your terminal in case you want to set this credential helper as the default

git config --global credential.helper store


In [ ]:
fakeNews = load_dataset("GonzaloA/fake_news")

Downloading:   0%|          | 0.00/1.03k [00:00<?, ?B/s]

Using custom data configuration GonzaloA--fake_news-ec767410c901c71e


Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

0 tables [00:00, ? tables/s]

0 tables [00:00, ? tables/s]

0 tables [00:00, ? tables/s]

Dataset parquet downloaded and prepared to /root/.cache/huggingface/datasets/GonzaloA___parquet/GonzaloA--fake_news-ec767410c901c71e/0.0.0/7328ef7ee03eaf3f86ae40594d46a1cec86161704e02dd19f232d81eee72ade8. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
modelCheckpoint="distilroberta-base"
tokenizer = AutoTokenizer.from_pretrained(modelCheckpoint)
print("El número total de tokens que tiene el tokenizador es de "+ str(tokenizer.vocab_size) + " tokens")

Downloading:   0%|          | 0.00/480 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

El número total de tokens que tiene el tokenizador es de 50265 tokens


In [ ]:
texto = "Hi, what's going on here?"
textoFragmentado = tokenizer.encode(texto)

for token in textoFragmentado:
  print(token, tokenizer.decode([token]))

textoCifrado = tokenizer(texto, return_tensors="pt")
print("\nUna vez tokenizado, la entrada que se envía al modelo es la siguiente: ")
print(str(textoCifrado))

0 <s>
30086 Hi
6 ,
99  what
18 's
164  going
15  on
259  here
116 ?
2 </s>

Una vez tokenizado, la entrada que se envía al modelo es la siguiente: 
{'input_ids': tensor([[    0, 30086,     6,    99,    18,   164,    15,   259,   116,     2]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}


In [ ]:
def tokenizeDataSets(dataset):
  return tokenizer(dataset["text"], truncation=True)

columnasDataSet = fakeNews["train"].column_names
columnasDataSet.remove("label")
print("Columnas que vamos a eliminar de la tokenización:\n" + str(columnasDataSet))

dataSetCifrado = fakeNews.map(tokenizeDataSets, batched=True, remove_columns=columnasDataSet)
print("Terminamos con el conjunto de datos cifrado y preparado para realizar el entrenamiento:\n" + str(dataSetCifrado))

Parameter 'function'=<function tokenizeDataSets at 0x7f3f2f7d8320> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


Columnas que vamos a eliminar de la tokenización:
['Unnamed: 0', 'title', 'text']


  0%|          | 0/9 [00:00<?, ?ba/s]

  0%|          | 0/25 [00:00<?, ?ba/s]

  0%|          | 0/9 [00:00<?, ?ba/s]

Terminamos con el conjunto de datos cifrado y preparado para realizar el entrenamiento:
DatasetDict({
    validation: Dataset({
        features: ['label', 'input_ids', 'attention_mask'],
        num_rows: 8117
    })
    train: Dataset({
        features: ['label', 'input_ids', 'attention_mask'],
        num_rows: 24353
    })
    test: Dataset({
        features: ['label', 'input_ids', 'attention_mask'],
        num_rows: 8117
    })
})


In [ ]:
nombreModelo = modelCheckpoint.split("/")[-1]
modeloPreentrenado = AutoModelForSequenceClassification.from_pretrained(modelCheckpoint, num_labels=2)
metrica = load_metric("accuracy")

Downloading:   0%|          | 0.00/316M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilroberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.decoder.weight', 'roberta.pooler.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at distilroberta-base and are newly initialized: ['classifier.dense.bias', 

In [ ]:
def compute_metrics(eval_preds):
  logits, labels = eval_preds
  predictions = np.argmax(logits, axis=-1)
  return metric.compute(predictions=predictions, references=labels)

In [ ]:
#Primera configuración

tamañoLotes = 16
numEpocasEntrenamiento = 1
numEjemplosEntrenamiento = 20_000
numEjemplosValidacion  = 5_000
dataSetEntrenamiento = dataSetCifrado["train"].shuffle(seed=42).select(range(numEjemplosEntrenamiento))
logging_steps = len(dataSetEntrenamiento) // (2*tamañoLotes * numEpocasEntrenamiento)
dataSetValidacion = dataSetCifrado["validation"].shuffle(seed=42).select(range(numEjemplosValidacion))
data_collator = DataCollatorWithPadding(tokenizer)

training_args1stconfig = TrainingArguments(
    output_dir = "results",
    num_train_epochs = numEpocasEntrenamiento,
    learning_rate = 2e-5,
    per_device_train_batch_size = tamañoLotes,
    per_device_eval_batch_size = tamañoLotes,
    load_best_model_at_end = True,
    metric_for_best_model = "accuracy",
    weight_decay = 0.01,
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    logging_steps = logging_steps,
    push_to_hub = True,
    hub_model_id = f"{nombreModelo}-finetuned-fakeNews"
)

Loading cached shuffled indices for dataset at /root/.cache/huggingface/datasets/GonzaloA___parquet/GonzaloA--fake_news-ec767410c901c71e/0.0.0/7328ef7ee03eaf3f86ae40594d46a1cec86161704e02dd19f232d81eee72ade8/cache-bebe15031583fb1b.arrow
Loading cached shuffled indices for dataset at /root/.cache/huggingface/datasets/GonzaloA___parquet/GonzaloA--fake_news-ec767410c901c71e/0.0.0/7328ef7ee03eaf3f86ae40594d46a1cec86161704e02dd19f232d81eee72ade8/cache-f9345c7905ef534d.arrow
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [ ]:
entrenador1st = Trainer(
    model = modeloPreentrenado,
    args = training_args1stconfig,
    train_dataset = dataSetEntrenamiento,
    eval_dataset = dataSetValidacion,
    data_collator = data_collator,
    tokenizer = tokenizer,
    compute_metrics = compute_metrics
)

Cloning https://huggingface.co/GonzaloA/distilroberta-base-finetuned-fakeNews into local empty directory.


Download file pytorch_model.bin:   0%|          | 3.47k/313M [00:00<?, ?B/s]

Download file runs/May07_07-53-31_d61dba157fb6/1651910210.7313166/events.out.tfevents.1651910210.d61dba157fb6.…

Download file runs/May06_14-43-29_232bc06bea9b/1651848434.5952466/events.out.tfevents.1651848434.232bc06bea9b.…

Download file runs/Jun17_09-56-40_133c4e44a298/events.out.tfevents.1655459908.133c4e44a298.76.0:  74%|#######3…

...

Clean file runs/May06_14-43-29_232bc06bea9b/events.out.tfevents.1651851445.232bc06bea9b.72.2:  24%|##3       |…

Download file training_args.bin: 100%|##########| 3.23k/3.23k [00:00<?, ?B/s]

Clean file runs/May07_07-53-31_d61dba157fb6/events.out.tfevents.1651910210.d61dba157fb6.71.0:  26%|##5       |…

Clean file training_args.bin:  31%|###       | 1.00k/3.23k [00:00<?, ?B/s]

Clean file pytorch_model.bin:   0%|          | 1.00k/313M [00:00<?, ?B/s]

In [ ]:
predictions = entrenador1st.predict(dataSetCifrado["validation"])
print(predictions.predictions.shape, predictions.label_ids.shape)

***** Running Prediction *****
  Num examples = 8117
  Batch size = 16


Epoch,Training Loss,Validation Loss


(8117, 2) (8117,)


In [ ]:
metric = load_metric("accuracy")
preds = np.argmax(predictions.predictions, axis = 1)
metric.compute(predictions=preds, references=predictions.label_ids)

{'accuracy': 0.9850930146605889}

In [ ]:
entrenador1st.train()

***** Running training *****
  Num examples = 20000
  Num Epochs = 1
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 1250


Epoch,Training Loss,Validation Loss,Accuracy
1,0.027800,0.033698,0.982800


***** Running Evaluation *****
  Num examples = 5000
  Batch size = 16


Epoch,Training Loss,Validation Loss


Saving model checkpoint to results/checkpoint-1250
Configuration saved in results/checkpoint-1250/config.json
Model weights saved in results/checkpoint-1250/pytorch_model.bin
tokenizer config file saved in results/checkpoint-1250/tokenizer_config.json
Special tokens file saved in results/checkpoint-1250/special_tokens_map.json
tokenizer config file saved in results/tokenizer_config.json
Special tokens file saved in results/special_tokens_map.json


Training completed. Do not forget to share your model on huggingface.co/models =)


Loading best model from results/checkpoint-1250 (score: 0.9828).


TrainOutput(global_step=1250, training_loss=0.029142758178710936, metrics={'train_runtime': 1084.1957, 'train_samples_per_second': 18.447, 'train_steps_per_second': 1.153, 'total_flos': 2649347973120000.0, 'train_loss': 0.029142758178710936, 'epoch': 1.0})

In [ ]:
#Segunda configuración

tamañoLotes = 16
numEpocasEntrenamiento = 1
numEjemplosEntrenamiento = 24_353
numEjemplosValidacion  = 8_117
dataSetEntrenamiento = dataSetCifrado["train"]
logging_steps = len(dataSetEntrenamiento) // (2*tamañoLotes * numEpocasEntrenamiento)
dataSetValidacion = dataSetCifrado["validation"]
data_collator = DataCollatorWithPadding(tokenizer)

training_args1stconfig = TrainingArguments(
    output_dir = "results",
    num_train_epochs = numEpocasEntrenamiento,
    learning_rate = 2e-5,
    per_device_train_batch_size = tamañoLotes,
    per_device_eval_batch_size = tamañoLotes,
    load_best_model_at_end = True,
    metric_for_best_model = "accuracy",
    weight_decay = 0.01,
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    logging_steps = logging_steps,
    push_to_hub = True,
    hub_model_id = f"{nombreModelo}-finetuned-fakeNews"
)

In [ ]:
entrenador2nd = Trainer(
    model = modeloPreentrenado,
    args = training_args2ndconfig,
    train_dataset = dataSetEntrenamiento,
    eval_dataset = dataSetValidacion,
    data_collator = data_collator,
    tokenizer = tokenizer,
    compute_metrics = compute_metrics
)

In [ ]:
predictions = entrenador2nd.predict(dataSetCifrado["validation"])
print(predictions.predictions.shape, predictions.label_ids.shape)

In [ ]:
metric = load_metric("accuracy")
preds = np.argmax(predictions.predictions, axis = 1)
metric.compute(predictions=preds, references=predictions.label_ids)

In [ ]:
entrenador2nd.train()

In [ ]:
#Tercera configuración

tamañoLotes = 16
numEpocasEntrenamiento = 3
numEjemplosEntrenamiento = 24_353
numEjemplosValidacion  = 8_117
dataSetEntrenamiento = dataSetCifrado["train"]
logging_steps = len(dataSetEntrenamiento) // (2*tamañoLotes * numEpocasEntrenamiento)
dataSetValidacion = dataSetCifrado["validation"]
data_collator = DataCollatorWithPadding(tokenizer)

training_args3rdconfig = TrainingArguments(
    output_dir = "results",
    num_train_epochs = numEpocasEntrenamiento,
    learning_rate = 2e-5,
    per_device_train_batch_size = tamañoLotes,
    per_device_eval_batch_size = tamañoLotes,
    load_best_model_at_end = True,
    metric_for_best_model = "accuracy",
    weight_decay = 0.01,
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    logging_steps = logging_steps,
    push_to_hub = True,
    hub_model_id = f"{nombreModelo}-finetuned-fakeNews"
)

In [ ]:
entrenador3rd = Trainer(
    model = modeloPreentrenado,
    args = training_args3rdconfig,
    train_dataset = dataSetEntrenamiento,
    eval_dataset = dataSetValidacion,
    data_collator = data_collator,
    tokenizer = tokenizer,
    compute_metrics = compute_metrics
)

In [ ]:
predictions = entrenador3rd.predict(dataSetCifrado["validation"])
print(predictions.predictions.shape, predictions.label_ids.shape)

In [ ]:
metric = load_metric("accuracy")
preds = np.argmax(predictions.predictions, axis = 1)
metric.compute(predictions=preds, references=predictions.label_ids)

In [ ]:
entrenador3rd.train()

In [ ]:
#Cuarta configuración

tamañoLotes = 16
numEpocasEntrenamiento = 3
numEjemplosEntrenamiento = 20_000
numEjemplosValidacion  = 5_000
dataSetEntrenamiento = dataSetCifrado["train"].shuffle(seed=42).select(range(numEjemplosEntrenamiento))
logging_steps = len(dataSetEntrenamiento) // (2*tamañoLotes * numEpocasEntrenamiento)
dataSetValidacion = dataSetCifrado["validation"].shuffle(seed=42).select(range(numEjemplosValidacion))
data_collator = DataCollatorWithPadding(tokenizer)

training_args4thconfig = TrainingArguments(
    output_dir = "results",
    num_train_epochs = numEpocasEntrenamiento,
    learning_rate = 2e-5,
    per_device_train_batch_size = tamañoLotes,
    per_device_eval_batch_size = tamañoLotes,
    load_best_model_at_end = True,
    metric_for_best_model = "accuracy",
    weight_decay = 0.01,
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    logging_steps = logging_steps,
    push_to_hub = True,
    hub_model_id = f"{nombreModelo}-finetuned-fakeNews"
)

Loading cached shuffled indices for dataset at /root/.cache/huggingface/datasets/GonzaloA___parquet/GonzaloA--fake_news-ec767410c901c71e/0.0.0/7328ef7ee03eaf3f86ae40594d46a1cec86161704e02dd19f232d81eee72ade8/cache-6ff244a52001ffc6.arrow
Loading cached shuffled indices for dataset at /root/.cache/huggingface/datasets/GonzaloA___parquet/GonzaloA--fake_news-ec767410c901c71e/0.0.0/7328ef7ee03eaf3f86ae40594d46a1cec86161704e02dd19f232d81eee72ade8/cache-668ac10f4e6c62b4.arrow
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [ ]:
entrenador = Trainer(
    model = modeloPreentrenado,
    args = training_args4thconfig,
    train_dataset = dataSetEntrenamiento,
    eval_dataset = dataSetValidacion,
    data_collator = data_collator,
    tokenizer = tokenizer,
    compute_metrics = compute_metrics
)

/content/results is already a clone of https://huggingface.co/GonzaloA/distilroberta-base-finetuned-fakeNews. Make sure you pull the latest changes with `repo.git_pull()`.


In [ ]:
predictions = entrenador.predict(dataSetCifrado["validation"])
print(predictions.predictions.shape, predictions.label_ids.shape)

***** Running Prediction *****
  Num examples = 8117
  Batch size = 16


KeyboardInterrupt: ignored

In [ ]:
metric = load_metric("accuracy")
preds = np.argmax(predictions.predictions, axis = 1)
metric.compute(predictions=preds, references=predictions.label_ids)

{'accuracy': 0.9843538253049157}

In [ ]:
entrenador.train()

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 20000
  Num Epochs = 3
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 3750


Epoch,Training Loss,Validation Loss,Accuracy
1,0.021600,0.051780,0.986400
2,0.016100,0.046462,0.987800
3,0.006500,0.049576,0.987800


***** Running Evaluation *****
  Num examples = 5000
  Batch size = 16


Saving model checkpoint to results/checkpoint-1250
Configuration saved in results/checkpoint-1250/config.json
Model weights saved in results/checkpoint-1250/pytorch_model.bin
tokenizer config file saved in results/checkpoint-1250/tokenizer_config.json
Special tokens file saved in results/checkpoint-1250/special_tokens_map.json
tokenizer config file saved in results/tokenizer_config.json
Special tokens file saved in results/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 5000
  Batch size = 16
Saving model checkpoint to results/checkpoint-2500
Configuration saved in results/checkpoint-2500/config.json
Model weights saved in results/checkpoint-2500/pytorch_model.bin
tokenizer config file saved in results/checkpoint-2500/tokenizer_config.json
Special tokens file saved in results/checkpoint-2500/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 5000
  Batch size = 16
Saving model checkpoint to results/checkpoint-3750
Configuration saved in resu

TrainOutput(global_step=3750, training_loss=0.021480363363027572, metrics={'train_runtime': 3296.0008, 'train_samples_per_second': 18.204, 'train_steps_per_second': 1.138, 'total_flos': 7947874195505472.0, 'train_loss': 0.021480363363027572, 'epoch': 3.0})

In [ ]:
entrenador.push_to_hub()

Saving model checkpoint to results
Configuration saved in results/config.json
Model weights saved in results/pytorch_model.bin
tokenizer config file saved in results/tokenizer_config.json
Special tokens file saved in results/special_tokens_map.json
Dropping the following result as it does not have all the necessary fields:
{'task': {'name': 'Text Classification', 'type': 'text-classification'}, 'metrics': [{'name': 'Accuracy', 'type': 'accuracy', 'value': 0.9878}]}


In [ ]:
model_checkpoint = "GonzaloA/distilroberta-base-finetuned-fakeNews"
pipe = pipeline("sentiment-analysis", model=model_checkpoint)

loading configuration file https://huggingface.co/GonzaloA/distilroberta-base-finetuned-fakeNews/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/5f38aef57f545922021a11568f9c2469c03563bb695ca255487f3279bf04daa8.842e764c15d519f6719510d6cf83485da0ba5b1a2d84457b0adb63e7dc57c55e
Model config RobertaConfig {
  "_name_or_path": "GonzaloA/distilroberta-base-finetuned-fakeNews",
  "architectures": [
    "RobertaForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 6,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "problem_type": "single_label_classification",
  "torch_dtype": "float32",
  "tra

In [ ]:
fakeNews["test"][2]

{'Unnamed: 0': 2,
 'label': 0,
 'text': 'With only three clowns left in GOP clown car, Republicans are doing everything they can to prevent Donald Trump from winning the Republican presidential nomination. During a segment on The Late Show, Stephen Colbert hilariously mocks Trump s  victory tantrum  and explains the only two ways to stop Trump from securing the nomination.Since the Republican primary race is being dominated by a has been reality TV star, Colbert appropriately likens the race to The Bachelor. Colbert lampoons Ohio governor John Kasich s claim that he can still win the nomination after his only win in Ohio. Colbert quips that Kasich would have to be made the governor of every remaining state in the U.S. in order for the candidate to have any real chance of winning the nomination.So, even though Kasich has not dropped out of the race, the only possible person left in the race who could defeat Trump is The Zodiac Killer Ted Cruz. That isn t good news for the GOP or the U.S

In [ ]:
pipe ("With only three clowns left in GOP clown car, Republicans are doing everything they can to prevent Donald Trump from winning the Republican presidential nomination. During a segment on The Late Show, Stephen Colbert hilariously mocks Trump s  victory tantrum  and explains the only two ways to stop Trump from securing the nomination.Since the Republican primary race is being dominated by a has been reality TV star, Colbert appropriately likens the race to The Bachelor. Colbert lampoons Ohio governor John Kasich s claim that he can still win the nomination after his only win in Ohio. Colbert quips that Kasich would have to be made the governor of every remaining state in the U.S. in order for the candidate to have any real chance of winning the nomination.So, even though Kasich has not dropped out of the race, the only possible person left in the race who could defeat Trump is The Zodiac Killer Ted Cruz. That isn t good news for the GOP or the U.S. Cruz, while he has yet to start a race riot at a campaign event, is actually far worse than Trump in terms of policy. Cruz is also incredibly creepy. Colbert jokes at the idea of Cruz winning the Republican nomination, saying: If every single Trump voter unites behind him, he could get enough delegates to be the nominee. Then the GOP establishment can say,  Goodbye, candidate we don t like! Hello, candidate, we can t stand! If Cruz cannot secure enough votes to secure the nomination, Republicans will have one final shot at defeating Trump   a brokered convention. So if Trump doesn t get to 1,237 delegates in the first round of voting, they re all free to vote for whoever. Which is great news. The Republican campaign can start all over again, only this time it s all crammed into one week. You can binge watch the death of the GOP! You can watch the segment, in full, below.Featured image from video screenshot")

[{'label': 'LABEL_0', 'score': 0.9999839067459106}]